In [ ]:
import sympy
from sympy import S, Symbol, symbols, simplify, nonlinsolve, Matrix, diff, factor, latex
from sympy.calculus.util import continuous_domain
from IPython.display import display, Latex

## Занятие 15
## Математический анализ
## Экстремум функции нескольких переменных
### Задание 1
Найти точки экстремума:

1) $z = x^2 + xy + y^2 - 2x - y$

2) $z = x^3y^2(6 - x - y)$, $(x, y > 0)$

3) $z = \frac{8}{x} + \frac{x}{y} + y$

4) $z = \frac{1 + x - y}{\sqrt{1 + x^2 + y^2}}$
###### Указание.
Вначале найти стационарные точки. Составить определитель из вторых производных в произвольной точке. Подставляя найденные (вещественные) стационарные точки, решить вопрос о наличии экстремума.

In [ ]:
# Функция для нахождения знака определителя матрицы определния точек экстремума
def Extr_Point_Det_Positive(func, x_v, y_v, x_cur, y_cur):
  matrix_det = Matrix([[func.diff(x_v, 2), func.diff(x_v, y_v)], [func.diff(x_v, y_v), func.diff(y_v, 2)]]).det().subs({x_v: x_cur, y_v: y_cur})
  return matrix_det > 0

# Функция для нахождения стационарных точек из поля вещественных чисел
def Get_Stat_Points(func, x_v, y_v, bug_case = False):
  if (not bug_case):
    stat_points = [point for point in nonlinsolve([func.diff(x_v), func.diff(y_v)], [x_v, y_v]) if all([coord.is_real for coord in point])]
  else:
    stat_points = [(point[x_v], point[y_v]) for point in sympy.solve([func.diff(x_v), func.diff(y_v)], x_v, y_v, dict=True, manual=True) if y_v in point.keys() ]
  cont_dom_x = continuous_domain(func, x_v, S.Reals)
  cont_dom_y = continuous_domain(func, y_v, S.Reals)
  stat_points_checked = [point for point in stat_points if (point[0] in cont_dom_x and point[1] in cont_dom_y)]
  return stat_points_checked

# Функция для нахождения точек экстремума функции
def Get_Extr_Points(func, x_v, y_v, shit_bug_case = False):
  lst_stat_points = Get_Stat_Points(func, x_v, y_v, shit_bug_case)
  lst_loc_max = []
  lst_loc_min = []
  for point in lst_stat_points:
    x_cur, y_cur = point
    if (Extr_Point_Det_Positive(func, x_v, y_v, x_cur, y_cur)):
      a = func.diff(x_v, 2).subs({x_v:x_cur, y_v:y_cur})
      b = func.diff(y_v, 2).subs({x_v:x_cur, y_v:y_cur})
      if (a > 0 or b > 0):
        lst_loc_min.append(point)
      elif (a < 0 or b < 0):
        lst_loc_max.append(point)
  if (len(lst_loc_max) == 0):
    lst_loc_max = None
  if (len(lst_loc_min) == 0):
    lst_loc_min = None
  return (lst_loc_min, lst_loc_max)


In [ ]:
# При вызове nonlinsolve для уравнения под номером 4, происходит баг со стороны метода nonlinsolve
# не имеющий решения(официального) на данный момент :D
x, y, z = sympy.symbols('x y z', real = True)
x_p, y_p = sympy.symbols('x y', positive = True, real = True)

lst_func_data = [(x**2 + x * y + y**2 - 2 * x - y, x, y), (x_p**3 * y_p**2 * (6 - x_p - y_p), x_p, y_p), (S(8)/x + S(x)/y + y, x, y), (S(1 + x - y)/(sympy.sqrt(1 + x**2 + y**2)), x, y)]
for i in range(len(lst_func_data)):
  func_cur, x_v_c, y_v_c = lst_func_data[i]
  shit_2_b_c = False
  if (i == 3 or i == 1):
    shit_2_b_c = True
  lst_min_cur, lst_max_cur = Get_Extr_Points(func_cur, x_v_c, y_v_c, shit_2_b_c)

  display(Latex(f"[{i + 1}]. \n"))
  display(Latex(fr"\text{{Func: }} {latex(sympy.Eq(z, func_cur))}"))
  display(Latex(fr"\text{{Стационарные точки : }} {latex(Get_Stat_Points(func_cur, x_v_c, y_v_c, shit_2_b_c))}"))
  display(Latex(fr"\text{{Точки локального минимума: }} {latex(lst_min_cur)}"))
  display(Latex(fr"\text{{Точки локального максимума: }} {latex(lst_max_cur)}"))

### Задание 2.
С помощью метода Лагранжа найти условный экстремум функции
$z = x + 2y$ при условии $x^2 + y^2 = 5$.

In [ ]:
x, y, lam = sympy.symbols("x y lambda")
dx, dy, dx2, dy2 = symbols('dx dy dx^2 dy^2')
lst_syms = [x, y, lam]
func_p_2 = x + 2 * y
res_func_2 = x**2 + y**2 - 5
func_2 = func_p_2 + lam * res_func_2

lst_stat_2 = [point for point in nonlinsolve([func_2.diff(var) for var in lst_syms], lst_syms) if all([coord.is_real for coord in point])]
diff_func_2 = sympy.factor(func_2.diff(x, 2) * dx2 + 2 * func_2.diff(x, y) * dx * dy + func_2.diff(y, 2) * dy2)

display(*[Latex(fr"{latex(sympy.Eq(lam, point[-1]))} \ : \ {latex(diff_func_2.subs({lam: point[-1]}))}") for point in lst_stat_2])
display(Latex(fr"\bullet \ При \  {latex(sympy.Eq(lam, 1/2))} \text{{ значение производной знакопостоянно и положительно }} {latex(lst_stat_2[0][:-1])} \text{{ - точка максимума}}"))
display(Latex(fr"\bullet \ При \  {latex(sympy.Eq(lam, -1/2))} \text{{ значение производной знакопостоянно и отрицательно }} {latex(lst_stat_2[1][:-1])} \text{{ - точка минимума}}"))

### Задание 3.
С помощью метода Лагранжа найти условный экстремум функции
$z=x^2+y^2$ при условии $\frac{x}{2}+\frac{y}{3}=1$.

In [ ]:
lst_syms = [x, y, lam]
func_p_3 = x**2 + y**2
res_func_3 = S(x)/2 + S(y)/3 - 1
func_3 = func_p_3 + lam * res_func_3

lst_stat_3 = [point for point in nonlinsolve([func_3.diff(var) for var in lst_syms], lst_syms) if all([coord.is_real for coord in point])]
diff_func_3 = sympy.factor(func_3.diff(x, 2) * dx2 + 2 * func_3.diff(x, y) * dx * dy + func_3.diff(y, 2) * dy2)

display(*[Latex(fr"{latex(sympy.Eq(lam, point[-1]))} \ : \ {latex(diff_func_3.subs({lam: point[-1]}))}") for point in lst_stat_3])
display(Latex(fr"\bullet \ При \  {latex(sympy.Eq(lam, S(-72)/13))} \text{{ значение производной знакопостоянно и положительно }} {latex(lst_stat_3[0][:-1])} \text{{ - точка максимума}}"))

### Задание 4.
С помощью метода Лагранжа найти экстремум функции $u = x - 2y + 2z$ при условии $x^2 + y^2 + z^2 = 9$.

In [ ]:
x, y, z, lam = sympy.symbols('x y z lambda')
dx, dy, dz, dx2, dy2, dz2 = symbols('dx dy dz dx^2 dy^2 dz^2')
lst_syms = [x, y, z, lam]
func_p_4 = x - 2 * y + 2 * z
res_func_4 = x**2 + y**2 + z**2 - 9
func_4 = func_p_4 + lam * res_func_4

lst_stat_4 = [point for point in nonlinsolve([func_4.diff(var) for var in lst_syms], lst_syms) if all([coord.is_real for coord in point])]

diff_func_4 = 0
diffs = {x: (dx, dx2), y: (dy, dy2), z: (dz, dz2)}
for i, var in enumerate(lst_syms[:-1]):
    diff_func_4+= func_4.diff(var, 2)*diffs[var][1]
    for j in range(i + 1, 3):
      diff_func_4 += 2*func_4.diff(var, lst_syms[j])*diffs[var][0]*diffs[lst_syms[j]][0]

display(*[Latex(fr"{latex(sympy.Eq(lam, point[-1]))} \ : \ {latex(diff_func_4.subs({lam: point[-1]}))}") for point in lst_stat_4])
display(Latex(fr"\bullet \ При \  {latex(sympy.Eq(lam, 1/2))} \text{{ значение производной знакопостоянно и положительно }} {latex(lst_stat_4[0][:-1])} \text{{ - точка максимума}}"))
display(Latex(fr"\bullet \ При \  {latex(sympy.Eq(lam, -1/2))} \text{{ значение производной знакопостоянно и отрицательно }} {latex(lst_stat_4[1][:-1])} \text{{ - точка минимума}}"))

### Индивидуальное задание
Найти точки экстремума $u = x^2 + y^2 + z^2 - xy + x - 2z$.

Варинат №134

\begin{align*}
    f(x, y, z) = - 19 x^{2} + 7 x y + 4 x z - 8 x - 14 y^{2} + 7 y z + 3 y + z^{2} + 6 z
\end{align*}

In [ ]:
# Функция для нахождения знака определителя матрицы определния точек экстремума для функции от 3х переменных
def Extr_Point_Det_Positive_3(func, x_v, y_v, z_v, x_cur, y_cur, z_cur):
  matrix_det = Matrix([[func.diff(x_v, 2), func.diff(x_v, y_v), func.diff(x_v, z_v)], [func.diff(y_v, x_v), func.diff(y_v, 2), func.diff(y_v, z_v)], [func.diff(z_v, x_v), func.diff(z_v, y_v), func.diff(z_v, 2)]]).det().subs({x_v: x_cur, y_v: y_cur, z_v: z_cur})
  return matrix_det > 0

# Функция для нахождения стационарных точек функции от 3х переменных из поля вещественных чисел
def Get_Stat_Points_3(func, x_v, y_v, z_v, bug_case = False):
  if (not bug_case):
    stat_points = [point for point in nonlinsolve([func.diff(x_v), func.diff(y_v), func.diff(z_v)], [x_v, y_v, z_v]) if all([coord.is_real for coord in point])]
  else:
    stat_points = [(point[x_v], point[y_v], point[z_v]) for point in sympy.solve([func.diff(x_v), func.diff(y_v), func.diff(z_v)], x_v, y_v, z_v, dict=True, manual=True) if all([sym in point.keys() for sym in [x_v, y_v, z_v]])]
  cont_dom_x = continuous_domain(func, x_v, S.Reals)
  cont_dom_y = continuous_domain(func, y_v, S.Reals)
  cont_dom_z = continuous_domain(func, z_v, S.Reals)
  stat_points_checked = [point for point in stat_points if (point[0] in cont_dom_x and point[1] in cont_dom_y and point[2] in cont_dom_z)]
  return stat_points_checked

# Функция для нахождения точек экстремума функции от 3х переменных
def Get_Extr_Points_3(func, x_v, y_v, z_v, shit_bug_case = False):
  lst_stat_points = Get_Stat_Points_3(func, x_v, y_v, z_v, shit_bug_case)
  lst_loc_max = []
  lst_loc_min = []
  for point in lst_stat_points:
    x_cur, y_cur, z_cur = point
    if (Extr_Point_Det_Positive_3(func, x_v, y_v, z_v, x_cur, y_cur, z_cur)):
      a = func.diff(x_v, 2).subs({x_v:x_cur, y_v:y_cur, z_v:z_cur})
      b = func.diff(y_v, 2).subs({x_v:x_cur, y_v:y_cur, z_v:z_cur})
      c = func.diff(z_v, 2).subs({x_v:x_cur, y_v:y_cur, z_v:z_cur})
      if (a > 0 or b > 0 or c > 0):
        lst_loc_min.append(point)
      elif (a < 0 or b < 0 or c < 0):
        lst_loc_max.append(point)
  if (len(lst_loc_max) == 0):
    lst_loc_max = None
  if (len(lst_loc_min) == 0):
    lst_loc_min = None
  return (lst_loc_min, lst_loc_max)

In [ ]:
x, y, z, u = sympy.symbols('x y z u', real = True)
func_ind = -19 * x**2 + 7 * x * y + 4 * x * z - 8 * x - 14 * y**2 + 7 * y * z + 3 * y + z**2 + 6 * z
lst_ind_min, lst_ind_max = Get_Extr_Points_3(func_ind, x, y, z)

display(Latex(fr"\text{{Func: }} {latex(sympy.Eq(u, func_ind))}"))
display(Latex(fr"\text{{Стационарные точки : }} {latex(Get_Stat_Points_3(func_ind, x, y, z))}"))
display(Latex(fr"\text{{Точки локального минимума: }} {latex(lst_ind_min)}"))
display(Latex(fr"\text{{Точки локального максимума: }} {latex(lst_ind_max)}"))